In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
from astropy.convolution import convolve, Box1DKernel
from scipy.interpolate import interp1d

import astropy.units as u
from astropy import time, coordinates as coord
import astropy.constants as const
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord
from matplotlib.gridspec import GridSpec


#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

I made a big table on MAST with all the IUE and HST data, did it work?

In [6]:
masttab = Table.read('MAST_Crossmatch_All_Observati.csv', format='csv')
masttab

TIC,RA,DEC,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_exptime,wavelength_region,filters,target_classification,t_obs_release,instrument_name,project,distance
str14,float64,float64,int64,str3,str9,str18,float64,float64,float64,float64,str7,str9,str32,float64,str13,str3,float64
TIC_381949122,76.37773198,-57.47270469,2,IUE,swp16862,HD 33262,76.3779942465,-57.4733210496,45089.51337,1859.864,UV,LOW DISP,--,nan,SWP,--,0.0
TIC_381949122,76.37773198,-57.47270469,2,IUE,swp16717,HD 33262,76.3774951703,-57.4733186916,45067.55881,1499.825,UV,LOW DISP,--,nan,SWP,--,0.0
TIC_381949122,76.37773198,-57.47270469,2,IUE,swp17006,HD 33262,76.3779942465,-57.4733210496,45110.43361,1859.864,UV,LOW DISP,--,nan,SWP,--,0.0
TIC_381949122,76.37773198,-57.47270469,2,IUE,swp16923,HD 33262,76.3779942465,-57.4733210496,45097.42542,1019.774,UV,LOW DISP,--,nan,SWP,--,0.0
TIC_30293512,198.0132679,-37.8030222,2,IUE,swp13987,HD 114613,198.0157392152,-37.8033803536,44742.23433,2099.48,UV,LOW DISP,--,nan,SWP,--,0.0
TIC_381949122,76.37773198,-57.47270469,2,IUE,swp16734,HD 33262,76.3774951703,-57.4733186916,45069.33122,1859.864,UV,LOW DISP,--,nan,SWP,--,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TIC_422478973,199.60130828,-18.31119382,3,HST,o6ic02070,HD115617,199.6007723699,-18.31165532888,52127.2209466782,1.0,Optical,G430L,STAR;G V-IV,52127.65092588,STIS/CCD,HST,0.430788289240312
TIC_422478973,199.60130828,-18.31119382,3,HST,o6ic020d0,HD115617,199.600772335,-18.31165537213,52127.2744074074,0.6,Optical,G750L,STAR;G V-IV,52127.67672444,STIS/CCD,HST,0.633862439315792


In [8]:
tics = np.unique(masttab['TIC'])
len(tics)

88

Want - number of stars with FUV data, NUV data, both, neither.

In [12]:
[print(flt) for flt in np.unique(masttab['filters'])]

E140H
E140M
E230H
E230M
ECH-A
ECH-B
G130M
G140L
G140M
G160M
G225M
G230L
G230LB
G230M
G230MB
G270M
G285M
G430L
G430M
G750L
G750M
HIGH DISP
LOW DISP
MIRROR-A2
MIRROR-N1
MIRROR-N2


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [39]:
fuv = 0
nuv = 0
both = 0
neither = 0

nuv_gratings = ['E230H', 'E230M','G225M', 'G230L','G230LB', 'G230M', 'G230MB','G270M','G285M', 'ECH-B']
fuv_gratings = ['E140H', 'E140M', 'G130M', 'G140L', 'G140M', 'G160M', 'ECH-A']

for i, tic in enumerate(tics):
    fuv_obs = False
    nuv_obs = False
    rows = masttab[np.where(masttab['TIC'] == tic)]
    if 'SWP' in rows['instrument_name']:
        fuv_obs = True
    if 'LWP' in  rows['instrument_name'] or 'LWR' in rows['instrument_name']:
        nuv_obs = True
        
    for grating in nuv_gratings:
        if grating in rows['filters']:
            nuv_obs = True
        
    for grating in fuv_gratings:
        if grating in rows['filters']:
            fuv_obs = True
    
    if nuv_obs:
        nuv +=1
    if fuv_obs:
        fuv +=1
    if nuv_obs and fuv_obs:
        both += 1
    if nuv_obs == False and fuv_obs == False:
        neither +=1
        # print(tic)
        # print(rows)
     
        
print(fuv)
print(nuv)    
print(both)
print(neither)
neither_all = 164 - both
print(neither_all)

55
72
41
2
123


In [35]:
# for row in masttab:
#     if row['filters'] in ['ECH-A', 'ECH-B']:
#         print(row['instrument_name'])

In [41]:
#164 total stars
55+72-41

86